# Project 4 - Train a Smartcab to Drive

## Implement a Basic Driving Agent

To begin, your only task is to get the smartcab to move around in the environment. At this point, you will not be concerned with any sort of optimal driving policy.

To complete this task, simply have your driving agent choose a random action from the set of possible actions (None, `'forward'`, `'left'`, `'right'`) at each intersection. Set the simulation deadline enforcement, **`enforce_deadline`** to **`False`** and observe how it performs.

### Code Snippet:
``` Python
def random_action(self):
    from random import choice
    return choice([None, 'forward', 'left', 'right'])
```

### Question 1:

**Observe what you see with the agent's behavior as it takes random actions. Does the smartcab eventually make it to the destination? Are there any other interesting observations to note?**

1. In almost 65% of the trials, the smartcab eventually makes it to the destination within the hard time limit (-100).
2. Other observations that were not mentioned in the problem statement:
    1. The agent seems to never reach the destination within the time limit (it always goes to negative time)
    2. Environment wrap around itself (East-most connected to West-most, and North-most connected to South-most.)
    3. Disobeying traffic lows results in -1 rewards, and the action is not performed
    4. Reaching the goal results in +10 rewards
    5. Performing a legal move results in either +2 or -0.5 (by looking at the code, I figured it is based on whether the smartcab followed the `next_waypoint` suggestion by the planner or not)
    
    

## Inform the Driving Agent

Now that your driving agent is capable of moving around in the environment, your next task is to identify a set of states that are appropriate for modeling the smartcab and environment.

The main source of state variables are the current inputs at the intersection, but not all may require representation. You may choose to explicitly define states, or use some combination of inputs as an implicit state.

At each time step, process the inputs and update the agent's current state using the `self.state` variable. Continue with the simulation deadline enforcement `enforce_deadline` being set to `False`, and observe how your driving agent now reports the change in state as the simulation progresses.

### Code Snippet

``` Python
def eval_state(self, next_waypoint, inputs):
legal_right = inputs['light'] == 'green' or inputs['left'] != 'forward'
legal_forward = inputs['light'] == 'green'
legal_left = inputs['light'] == 'green' and inputs['oncoming'] != 'right' and inputs['oncoming'] != 'forward'
return {'next_waypoint': next_waypoint, 
        'legal_right': legal_right,
        'legal_forward': legal_forward,
        'legal_left': legal_left}
```

### Question 2:

**What states have you identified that are appropriate for modeling the smartcab and environment? Why do you believe each of these states to be appropriate for this problem?**

1. I was able to identify four states that would model the smarcab and the environment:
    1. `next_waypoint`: To represent the direction suggested by the planner
    2. `legal_right`: To represent whether it is legal to turn right at the intersection
    3. `legal_forward`: To represent whether it is legal to keep moving forward at the intersection
    4. `legal_left`: To represent whether it is legal to turn left at the intersection
    
2. Appropriateness of these states were achieved by:
    1. `next_waypoint`: Since the `next_waypoint` affects the reward that would be achieved at each state, it is important to have it as part of our state as a representation of the smartcab planner
    2. `legal_right`, `legal_forward`, and `legal_left`: These three variables embed the information received by the sensors of the smartcab. Instead of using sensor data directly (`light`, `right`, `oncoming`, and `left`), these three states allow us to achieve:
        1. Reduce the state-space representing the enironment from $2 x 3 x 3 x 3 = 54$ (`Light = ['green', 'red']`, `right = oncoming = left = ['right', 'forward', 'left']`. Note that dummy agents don't have `'None'` as an option) to $2 x 2 x 2 = 8$ (`legal_right = legal_forward = legal_left = [True, False]`)
        2. While reducing the state-space, the information needed for the smartcab to make its decision is still preserved. The value of interest behind knowing the sensor data for an intersection is to know whether a right, left, or forward move is legal. These three states keep that information. In fact, they represent this information.
        

### Question 3:

**How many states in total exist for the smartcab in this environment? Does this number seem reasonable given that the goal of Q-Learning is to learn and make informed decisions about each state? Why or why not?**

1. Number of states that exist are $2 x 2 x 2 x 3 = 24$ (`legal_right`, `legal_forward`, and `legal_left` have two possible values, `True` and `False`. `next_waypoint` has three possible values, `right`, `forward`, and `left`. Note that the planner doesn't give `None` as an option).

2. The number of states seems reasonable for a Q-Learning since it is low enough. These 24 states could easily be traversed several times by running the simulation several times. 

## Implement a Q-Learning Driving Agent

With your driving agent being capable of interpreting the input information and having a mapping of environmental states, your next task is to implement the Q-Learning algorithm for your driving agent to choose the best action at each time step, based on the Q-values for the current state and action.

Each action taken by the smartcab will produce a reward which depends on the state of the environment. The Q-Learning driving agent will need to consider these rewards when updating the Q-values. Once implemented, set the simulation deadline enforcement `enforce_deadline` to `True`. Run the simulation and observe how the smartcab moves about the environment in each trial.

### Code Snippet

``` Python
def Q_action(self, state):
    actions = []
    best_score = self.Q.get((str(state), None), self.default_Q_value)
    for action in [None, 'forward', 'left', 'right']:
        val = self.Q.get((str(state), action), self.default_Q_value)
        if val < best_score - 1e-9:
            # Better action was found before
            continue
        elif val > best_score + 1e-9:
            # This is a better action than the one(s) found before
            actions = [action]
            best_score = val
        else:
            # This action produces the same best_score so far
            actions.append(action)

    from random import choice
    return choice(actions)

def explore_exploite_action(self):
    from random import random
    if random() < self.epsilon:
        # Exploration
        print "LearningAgent.explore_exploite_action(): Exploration" # [debug]
        return self.random_action()
    else:
        # Exploitation
        print "LearningAgent.explore_exploite_action(): Exploitation" # [debug]
        return self.Q_action(self.state)

def bellman(self, state, action, reward):
    new_next_waypoint = self.planner.next_waypoint()
    new_inputs = self.env.sense(self)
    new_state = self.eval_state(new_next_waypoint, new_inputs)
    new_action = self.Q_action(new_state)

    old_val = self.Q.get((str(state), action), self.default_Q_value)
    update_val = reward + self.gamma * self.Q.get((str(new_state), new_action), self.default_Q_value)

    return (1 - self.alpha) * old_val + self.alpha * update_val

def update(self, t):
    # Gather inputs
    self.next_waypoint = self.planner.next_waypoint()  # from route planner, also displayed by simulator
    inputs = self.env.sense(self)
    deadline = self.env.get_deadline(self)

    # TODO: Update state
    self.state = self.eval_state(self.next_waypoint, inputs)

    # TODO: Select action according to your policy
    action = self.explore_exploite_action()

    # Execute action and get reward
    reward = self.env.act(self, action)

    # TODO: Learn policy based on state, action, reward
    self.Q[(str(self.state), action)] = self.bellman(self.state, action, reward)
```

### Question 4:

**What changes do you notice in the agent's behavior when compared to the basic driving agent when random actions were always taken? Why is this behavior occurring?**

1. One major difference between the original behavior and the new one is that it can successfully reach the goal in more than 90% of the cases. Also, after the initial trials, the agent learns to obey the traffic law and the planner suggestions.

2. This behavior is emerging because the agent learns that obeying the law would result in more reward in the long run. Also, it learns that the planner takes it to the destination where even more reward is achieved. On the other hand, the random actions were resulting in negative rewards in several cases.

## Improve the Q-Learning Driving Agent

Your final task for this project is to enhance your driving agent so that, after sufficient training, the smartcab is able to reach the destination within the allotted time safely and efficiently. 

Parameters in the Q-Learning algorithm, such as the learning rate (**`alpha`**), the discount factor (**`gamma`**) and the exploration rate (**`epsilon`**) all contribute to the driving agent’s ability to learn the best action for each state.

To improve on the success of your smartcab:

- Set the number of trials, `n_trials`, in the simulation to 100.
- Run the simulation with the deadline enforcement `enforce_deadline` set to `True` (you will need to reduce the update delay `update_delay` and set the `display` to `False`).
- Observe the driving agent’s learning and smartcab’s success rate, particularly during the later trials.
- Adjust one or several of the above parameters and iterate this process.

This task is complete once you have arrived at what you determine is the best combination of parameters required for your driving agent to learn successfully.

### Question 5:

**Report the different values for the parameters tuned in your basic implementation of Q-Learning. For which set of parameters does the agent perform best? How well does the final driving agent perform?**

For `alpha`, `gamma`, and `epsilon`, I used the following values: `[0.1, 0.5, 0.9]`. Using these values, the code provided above was run over 100 trials and the success rate is reported in the following tables:

<table>
  <tr>
    <th></th>
    <th colspan="3">Epsilon=0.1</th>
  </tr>
  <tr>
    <td><br></td>
    <td>Gamma=0.1</td> <td>Gamma=0.5</td> <td>Gamma=0.9</td>
  </tr>
  <tr>
    <td>Alpha=0.1</td>
    <td>0.99<br></td> <td>0.92<br></td> <td>0.90<br></td>
  </tr>
  <tr>
    <td>Alpha=0.5</td>
    <td>0.98<br></td> <td>0.90<br></td> <td>0.88</td>
  </tr>
  <tr>
    <td>Alpha=0.9</td>
    <td>0.96<br></td> <td>0.94<br></td> <td>0.82</td>
  </tr>
</table>

<table>
  <tr>
    <th></th>
    <th colspan="3">Epsilon=0.5</th>
  </tr>
  <tr>
    <td><br></td>
    <td>Gamma=0.1</td> <td>Gamma=0.5</td> <td>Gamma=0.9</td>
  </tr>
  <tr>
    <td>Alpha=0.1</td>
    <td>0.70<br></td> <td>0.60</td> <td>0.54</td>
  </tr>
  <tr>
    <td>Alpha=0.5</td>
    <td>0.64</td> <td>0.54</td> <td>0.49</td>
  </tr>
  <tr>
    <td>Alpha=0.9</td>
    <td>0.73</td> <td>0.55</td> <td>0.45</td>
  </tr>
</table>

<table>
  <tr>
    <th></th>
    <th colspan="3">Epsilon=0.9</th>
  </tr>
  <tr>
    <td><br></td>
    <td>Gamma=0.1</td> <td>Gamma=0.5</td> <td>Gamma=0.9</td>
  </tr>
  <tr>
    <td>Alpha=0.1</td>
    <td>0.27</td> <td>0.24</td> <td>0.24</td>
  </tr>
  <tr>
    <td>Alpha=0.5</td>
    <td>0.21</td>  <td>0.23</td> <td>0.22</td>
  </tr>
  <tr>
    <td>Alpha=0.9</td>
    <td>0.32</td> <td>0.26</td> <td>0.24</td>
  </tr>
</table>

As we can see from the tables above. The optimal values for `alpha`, `gamma`, and `epsilon` are `(alpha = 0.1, gamma = 0.1, epsilon = 0.1)` with success rate 0.99

### Question 6:

**Does your agent get close to finding an optimal policy, i.e. reach the destination in the minimum possible time, and not incur any penalties? How would you describe an optimal policy for this problem?**

Yes. By looking at value for Q, the optimal policy could be extracted as follow:

1. As long as the `next_waypoint` is giving a legal move (Ex: `next_waypoint = left` and `legal_left = true`), the agent follows executes the `next_waypoint` action.
2. When the `next_waypoint` is an illegal move, the agent should execute the `None` action (i.e. stay in its place)

Q Values from one of the runs:

```
("{'legal_forward': False, 'next_waypoint': 'forward', 'legal_right': False, 'legal_left': False}", 'forward') -0.1

("{'legal_forward': False, 'next_waypoint': 'forward', 'legal_right': True, 'legal_left': False}", None) 0.0
("{'legal_forward': False, 'next_waypoint': 'forward', 'legal_right': True, 'legal_left': False}", 'forward') -0.86491482
("{'legal_forward': False, 'next_waypoint': 'forward', 'legal_right': True, 'legal_left': False}", 'left') -0.87842334540
("{'legal_forward': False, 'next_waypoint': 'forward', 'legal_right': True, 'legal_left': False}", 'right') -0.2690066011

("{'legal_forward': False, 'next_waypoint': 'left', 'legal_right': True, 'legal_left': False}", None) 0.0
("{'legal_forward': False, 'next_waypoint': 'left', 'legal_right': True, 'legal_left': False}", 'forward') -0.3439
("{'legal_forward': False, 'next_waypoint': 'left', 'legal_right': True, 'legal_left': False}", 'left') -0.3439
("{'legal_forward': False, 'next_waypoint': 'left', 'legal_right': True, 'legal_left': False}", 'right') -0.129971475844

("{'legal_forward': False, 'next_waypoint': 'right', 'legal_right': True, 'legal_left': False}", None) 0.0567332846388
("{'legal_forward': False, 'next_waypoint': 'right', 'legal_right': True, 'legal_left': False}", 'forward') -0.2975459219
("{'legal_forward': False, 'next_waypoint': 'right', 'legal_right': True, 'legal_left': False}", 'left') -0.293771039618
("{'legal_forward': False, 'next_waypoint': 'right', 'legal_right': True, 'legal_left': False}", 'right') 3.18236699359

("{'legal_forward': True, 'next_waypoint': 'forward', 'legal_right': True, 'legal_left': True}", None) 0.312506404171
("{'legal_forward': True, 'next_waypoint': 'forward', 'legal_right': True, 'legal_left': True}", 'forward') 3.48707180223
("{'legal_forward': True, 'next_waypoint': 'forward', 'legal_right': True, 'legal_left': True}", 'left') -0.113783784226
("{'legal_forward': True, 'next_waypoint': 'forward', 'legal_right': True, 'legal_left': True}", 'right') -0.199865794218

("{'legal_forward': True, 'next_waypoint': 'left', 'legal_right': True, 'legal_left': True}", None) 0.0565135031121
("{'legal_forward': True, 'next_waypoint': 'left', 'legal_right': True, 'legal_left': True}", 'left') 3.14713894324
("{'legal_forward': True, 'next_waypoint': 'left', 'legal_right': True, 'legal_left': True}", 'right') -0.0705100757644

("{'legal_forward': True, 'next_waypoint': 'right', 'legal_right': True, 'legal_left': True}", None) 0.0571260333905
("{'legal_forward': True, 'next_waypoint': 'right', 'legal_right': True, 'legal_left': True}", 'forward') -0.059418868388
("{'legal_forward': True, 'next_waypoint': 'right', 'legal_right': True, 'legal_left': True}", 'left') -0.033492789082
("{'legal_forward': True, 'next_waypoint': 'right', 'legal_right': True, 'legal_left': True}", 'right') 3.08395691024
```